In [11]:
import torch
import torch.nn as nn


torch.manual_seed(0) #set rand seed same each time for debugging


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device) #check if we can use cuda "hoepfully we can for actual car"

N = 10000  #set 10k pairs
X = (torch.rand(N, 2) * 20 - 10).to(device)  # works with -10,10 "much better than just pos"
y = X.sum(dim=1, keepdim=True)               # true sum [10k,1]

model = nn.Linear(2, 1, bias=False).to(device) #2 inp 1 out neural nw no bias because pure add b should be 0

loss_fn = nn.MSELoss()  #loss uses mse to see how far off we are "punish"

optimizer = torch.optim.Adam(model.parameters(), lr=0.1) #adam optimizes weights based on gradients

#training
EPOCHS = 200
for epoch in range(1, EPOCHS + 1):
    #grab predictions from model
    preds = model(X)
    loss = loss_fn(preds, y)  # compare to wha tit should be

    # Backpropagation gradient descent
    optimizer.zero_grad() # reset prev gradients
    loss.backward()    #calculate new gradients
    optimizer.step()       # update weights

    if epoch % 50 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.8f}") #print every 50 epoch to track


print("Learned weights:", model.weight.detach().cpu().numpy()) #check model

#test
def predict(a, b):
    with torch.no_grad():  # dont track gradients for testing
        inp = torch.tensor([[float(a), float(b)]], device=device)
        out = model(inp).item()
    print(f"{a} + {b} = predicted {out:.4f} (actual {a+b:.4f})")

predict(6, 7)
predict(10, -2.5)
predict(-8.3, 1.2)
predict(0, 0)
predict(1, 10.0)


Using device: cpu
Epoch 001 | Loss: 87.06085968
Epoch 050 | Loss: 0.28439713
Epoch 100 | Loss: 0.00070914
Epoch 150 | Loss: 0.00000367
Epoch 200 | Loss: 0.00000006
Learned weights: [[0.99998283 1.000037  ]]
6 + 7 = predicted 13.0002 (actual 13.0000)
10 + -2.5 = predicted 7.4997 (actual 7.5000)
-8.3 + 1.2 = predicted -7.0998 (actual -7.1000)
0 + 0 = predicted 0.0000 (actual 0.0000)
1 + 10.0 = predicted 11.0004 (actual 11.0000)
